In [36]:
from stores.game_store import GameStore
from constants.lang import ENGLISH, RUSSIAN
from stores.trade_store import TradeStore

import pandas as pd

In [37]:
gs = GameStore(RUSSIAN)
ref_gs = GameStore(ENGLISH)
ts = TradeStore(RUSSIAN)
ref_ts = TradeStore(ENGLISH)

In [38]:
from constants.filenames import ARMOUR_TYPES, BASE_ITEM_TYPES, ITEM_CLASS_CATEGORIES, ITEM_CLASSES, SKILL_GEMS, TAGS, WEAPON_TYPES, WORDS


base_types = gs.get(BASE_ITEM_TYPES)[["Name"]]
item_class = ref_gs.get(ITEM_CLASSES)
item_category = ref_gs.get(ITEM_CLASS_CATEGORIES)
ref_base_types = ref_gs.get(BASE_ITEM_TYPES)[["_index", "ItemClass", "Width", "Height", "Name", "DropLevel", "Tags"]].rename({"Name": "refName", "_index": "internal_index"}, axis=1)
words = gs.get(WORDS)
armour = gs.get(ARMOUR_TYPES)
weapon = gs.get(WEAPON_TYPES)
skill_gem = gs.get(SKILL_GEMS)
tags = gs.get(TAGS)
tags_lookup = tags["Id"].to_dict()

In [39]:
joined_base = base_types.join(ref_base_types, validate="1:1").rename({"refName": "type"}, axis=1)
joined_base

,Name,internal_index,ItemClass,Width,Height,type,DropLevel,Tags
0,Точильный камень,0,30,1,1,Blacksmith's Whetstone,5,[348]
1,Резец чародея,1,30,1,1,Arcanist's Etcher,5,[348]
2,Свиток мудрости,2,30,1,1,Scroll of Wisdom,1,[]
3,Сфера хаоса,3,30,1,1,Chaos Orb,12,[]
4,Большая сфера хаоса,4,30,1,1,Greater Chaos Orb,72,[]
...,...,...,...,...,...,...,...,...
4055,Учётный журнал начальника тюрьмы,4055,31,2,2,Warden's Ledger,1,[1045]
4056,Книга специализации,4056,31,2,2,Book of Specialisation,1,[1045]
4057,Книга специализации,4057,31,2,2,Book of Specialisation,1,[1045]
4058,Дар Расплавленного,4058,31,2,2,Molten One's Gift,1,[1045]


In [40]:


ts_items = ref_ts.items()

unique_mask = ts_items["unique"] == True
non_unique_mask = ts_items["unique"] == False

unique_items = ts_items.loc[unique_mask].copy()
non_unique_items = ts_items.loc[non_unique_mask].copy()

unique_items["namespace"] = "UNIQUE"
non_unique_items["namespace"] = non_unique_items["id"].apply(lambda x: "GEM" if x == "gem" else "ITEM")
non_unique_items.loc[non_unique_items["type"].str.startswith("Uncut"), "namespace"] = "ITEM"
non_unique_items

,id,label,type,text,name,unique,namespace
0,accessory,Accessories,Crimson Amulet,None,None,False,ITEM
1,accessory,Accessories,Gold Amulet,None,None,False,ITEM
2,accessory,Accessories,Pearlescent Amulet,None,None,False,ITEM
3,accessory,Accessories,Azure Amulet,None,None,False,ITEM
4,accessory,Accessories,Amber Amulet,None,None,False,ITEM
...,...,...,...,...,...,...,...
2399,sanctum,Sanctum Research,Vase Relic,None,None,False,ITEM
2400,sanctum,Sanctum Research,Seal Relic,None,None,False,ITEM
2401,sanctum,Sanctum Research,Coffer Relic,None,None,False,ITEM
2402,sanctum,Sanctum Research,Tapestry Relic,None,None,False,ITEM


In [41]:
# Edge case fix: Rename specific items
outdated_names = {
    "Sekhema's Resolve": "Sekhema's Resolve Fire",
    "The Road Warrior": "The Immortan",
    "Byrnabas": "Brynabas",
    "Splinter of Lorrata": "Splinter of Loratta"
}
unique_items["name"] = unique_items["name"].replace(outdated_names)
unique_items.loc[unique_items["type"] == "Hardwood Spear"]


,id,label,type,text,name,unique,namespace
2374,weapon,Weapons,Hardwood Spear,Splinter of Lorrata Hardwood Spear,Splinter of Loratta,True,UNIQUE


In [42]:
from constants.known_stats import UNIQUE_ITEMS_FIXED_STATS


named_unique_items = unique_items.merge(words[["Text", "Text2"]], left_on="name", right_on="Text", how="left")[["type", "namespace", "Text2", "Text"]].rename({"Text2": "name", "Text": "refName"}, axis=1)
named_unique_items["unique"] = named_unique_items.apply(
    lambda row: {"base": row["type"]} if row["refName"] not in UNIQUE_ITEMS_FIXED_STATS else {"base": row["type"], "fixedStats": UNIQUE_ITEMS_FIXED_STATS[row["refName"]]}, axis=1
)

named_unique_items.loc[named_unique_items["refName"] == "Darkness Enthroned"]

,type,namespace,name,refName,unique
15,Fine Belt,UNIQUE,Воцарившаяся тьма,Darkness Enthroned,"{'base': 'Fine Belt', 'fixedStats': ['#% incre..."


In [43]:
classes = item_class[["_index", "ItemClassCategory"]].merge(item_category.rename({"_index":"_index2"}, axis=1)[["_index2", "Id"]], left_on="ItemClassCategory", right_on="_index2", how="left")
combined_non_unique = joined_base.merge(right=non_unique_items[["type", "namespace"]], on="type", how="right").rename({"type": "refName", "Name":"name"}, axis=1)
combined_non_unique = combined_non_unique.merge(classes[["_index", "Id"]], left_on="ItemClass", right_on="_index", validate="m:1").drop(["_index"], axis=1)
combined_non_unique

,name,internal_index,ItemClass,Width,Height,refName,DropLevel,Tags,namespace,Id
0,Алый амулет,2381,4,1,1,Crimson Amulet,1,[],ITEM,Amulet
1,Золотой амулет,2390,4,1,1,Gold Amulet,35,[],ITEM,Amulet
2,Амулет с перламутром,2391,4,1,1,Pearlescent Amulet,8,[],ITEM,Amulet
3,Лазурный амулет,2382,4,1,1,Azure Amulet,1,[],ITEM,Amulet
4,Амулет с янтарём,2383,4,1,1,Amber Amulet,8,[],ITEM,Amulet
...,...,...,...,...,...,...,...,...,...,...
1994,Ваза,1826,84,1,4,Vase Relic,65,[984],ITEM,Relic
1995,Печать,1827,84,2,1,Seal Relic,1,[982],ITEM,Relic
1996,Ларец,1828,84,2,2,Coffer Relic,78,[984],ITEM,Relic
1997,Гобелен,1829,84,3,1,Tapestry Relic,1,[983],ITEM,Relic


In [44]:
def create_armour_dict(row):
    armour_dict = {}
    if row['Armour'] != 0:
        armour_dict['ar'] = [row['Armour'], row['Armour']]
    if row['Evasion'] != 0:
        armour_dict['ev'] = [row['Evasion'], row['Evasion']]
    if row['EnergyShield'] != 0:
        armour_dict['es'] = [row['EnergyShield'], row['EnergyShield']]
    return armour_dict

armour["armour"] = armour.apply(create_armour_dict, axis=1)
ar_filtered = armour[["BaseItemType", "armour"]]
ar_filtered

,BaseItemType,armour
0,338,{}
1,339,{}
2,340,{}
3,341,"{'ar': [51, 51], 'ev': [44, 44], 'es': [21, 21]}"
4,342,{}
...,...,...
555,2805,"{'ev': [144, 144]}"
556,2806,"{'es': [52, 52]}"
557,2807,"{'es': [58, 58]}"
558,2808,"{'es': [63, 63]}"


In [45]:
plus_ar = combined_non_unique.merge(ar_filtered, left_on="internal_index", right_on="BaseItemType", how="left").drop("BaseItemType", axis=1)
plus_ar.loc[plus_ar["armour"].notna()]

,name,internal_index,ItemClass,Width,Height,refName,DropLevel,Tags,namespace,Id,armour
44,Кожаный жилет,2097,24,2,3,Leather Vest,1,"[39, 1075]",ITEM,Body Armour,"{'ev': [30, 30]}"
45,Куртка контрабандиста,2106,24,2,3,Smuggler Coat,51,"[39, 1165]",ITEM,Body Armour,"{'ev': [271, 271]}"
46,Плащ корсара,2699,24,2,3,Corsair Coat,80,"[39, 1165]",ITEM,Body Armour,"{'ev': [406, 406]}"
47,Жилет наездника,2107,24,2,3,Strider Vest,52,[39],ITEM,Body Armour,"{'ev': [275, 275]}"
48,Бригантный доспех,2111,24,2,3,Armoured Vest,73,[39],ITEM,Body Armour,"{'ev': [406, 406]}"
...,...,...,...,...,...,...,...,...,...,...,...
532,Угрожающий гербовый щит,2679,26,2,3,Glowering Crest Shield,62,"[42, 229]",ITEM,Shield,"{'ar': [80, 80], 'es': [28, 28]}"
533,Гербовый щит ваал,2800,26,2,3,Vaal Crest Shield,75,"[42, 229]",ITEM,Shield,"{'ar': [100, 100], 'es': [35, 35]}"
534,Неуступчивый гербовый щит,2339,26,2,3,Wayward Crest Shield,45,"[42, 229, 1165]",ITEM,Shield,"{'ar': [61, 61], 'es': [22, 22]}"
535,Гербовый щит из чёрной стали,2801,26,2,3,Blacksteel Crest Shield,80,"[42, 229, 1165]",ITEM,Shield,"{'ar': [109, 109], 'es': [37, 37]}"


In [46]:
plus_gem  = plus_ar.copy()
plus_gem["gem"] = plus_gem["namespace"].apply(
    lambda x: {"awakened": False, "transfigured": False} if x == "GEM" else None
)
plus_gem

,name,internal_index,ItemClass,Width,Height,refName,DropLevel,Tags,namespace,Id,armour,gem
0,Алый амулет,2381,4,1,1,Crimson Amulet,1,[],ITEM,Amulet,NaN,None
1,Золотой амулет,2390,4,1,1,Gold Amulet,35,[],ITEM,Amulet,NaN,None
2,Амулет с перламутром,2391,4,1,1,Pearlescent Amulet,8,[],ITEM,Amulet,NaN,None
3,Лазурный амулет,2382,4,1,1,Azure Amulet,1,[],ITEM,Amulet,NaN,None
4,Амулет с янтарём,2383,4,1,1,Amber Amulet,8,[],ITEM,Amulet,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1994,Ваза,1826,84,1,4,Vase Relic,65,[984],ITEM,Relic,NaN,None
1995,Печать,1827,84,2,1,Seal Relic,1,[982],ITEM,Relic,NaN,None
1996,Ларец,1828,84,2,2,Coffer Relic,78,[984],ITEM,Relic,NaN,None
1997,Гобелен,1829,84,3,1,Tapestry Relic,1,[983],ITEM,Relic,NaN,None


In [47]:
combined_unique = joined_base.merge(named_unique_items, on="type", how="right").drop("Name", axis=1)
combined_unique

,internal_index,ItemClass,Width,Height,type,DropLevel,Tags,namespace,name,refName,unique
0,2401,5,1,1,Gold Ring,40,[],UNIQUE,Андвариус,Andvarius,{'base': 'Gold Ring'}
1,2388,4,1,1,Stellar Amulet,25,[],UNIQUE,Астраментис,Astramentis,{'base': 'Stellar Amulet'}
2,2389,4,1,1,Solar Amulet,30,[],UNIQUE,Сигнал Азиса,Beacon of Azis,{'base': 'Solar Amulet'}
3,2411,21,2,1,Double Belt,44,[],UNIQUE,Цацконосец,Bijouborne,{'base': 'Double Belt'}
4,2406,21,2,1,Wide Belt,14,[],UNIQUE,Пряжка первородства,Birthright Buckle,{'base': 'Wide Belt'}
...,...,...,...,...,...,...,...,...,...,...,...
450,1827,84,2,1,Seal Relic,1,[982],UNIQUE,Сменяющиеся сезоны,The Changing Seasons,{'base': 'Seal Relic'}
451,1826,84,1,4,Vase Relic,65,[984],UNIQUE,Отчаянный союз,The Desperate Alliance,{'base': 'Vase Relic'}
452,1830,84,4,1,Incense Relic,65,[984],UNIQUE,Последнее пламя,The Last Flame,{'base': 'Incense Relic'}
453,1825,84,1,3,Amphora Relic,1,[983],UNIQUE,Вытяжка миротворца,The Peacemaker's Draught,{'base': 'Amphora Relic'}


In [48]:
combined = pd.concat([combined_non_unique, combined_unique]).reset_index(drop=True)
combined

,name,internal_index,ItemClass,Width,Height,refName,DropLevel,Tags,namespace,Id,type,unique
0,Алый амулет,2381,4,1,1,Crimson Amulet,1,[],ITEM,Amulet,NaN,NaN
1,Золотой амулет,2390,4,1,1,Gold Amulet,35,[],ITEM,Amulet,NaN,NaN
2,Амулет с перламутром,2391,4,1,1,Pearlescent Amulet,8,[],ITEM,Amulet,NaN,NaN
3,Лазурный амулет,2382,4,1,1,Azure Amulet,1,[],ITEM,Amulet,NaN,NaN
4,Амулет с янтарём,2383,4,1,1,Amber Amulet,8,[],ITEM,Amulet,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2449,Сменяющиеся сезоны,1827,84,2,1,The Changing Seasons,1,[982],UNIQUE,NaN,Seal Relic,{'base': 'Seal Relic'}
2450,Отчаянный союз,1826,84,1,4,The Desperate Alliance,65,[984],UNIQUE,NaN,Vase Relic,{'base': 'Vase Relic'}
2451,Последнее пламя,1830,84,4,1,The Last Flame,65,[984],UNIQUE,NaN,Incense Relic,{'base': 'Incense Relic'}
2452,Вытяжка миротворца,1825,84,1,3,The Peacemaker's Draught,1,[983],UNIQUE,NaN,Amphora Relic,{'base': 'Amphora Relic'}


In [49]:
combined_filtered = combined.drop(["ItemClass", "type", "DropLevel"], axis=1).rename({"Id": "category", "Width": "w", "Height": "h", "Tags": "tags"}, axis=1)
combined_filtered

,name,internal_index,w,h,refName,tags,namespace,category,unique
0,Алый амулет,2381,1,1,Crimson Amulet,[],ITEM,Amulet,NaN
1,Золотой амулет,2390,1,1,Gold Amulet,[],ITEM,Amulet,NaN
2,Амулет с перламутром,2391,1,1,Pearlescent Amulet,[],ITEM,Amulet,NaN
3,Лазурный амулет,2382,1,1,Azure Amulet,[],ITEM,Amulet,NaN
4,Амулет с янтарём,2383,1,1,Amber Amulet,[],ITEM,Amulet,NaN
...,...,...,...,...,...,...,...,...,...
2449,Сменяющиеся сезоны,1827,2,1,The Changing Seasons,[982],UNIQUE,NaN,{'base': 'Seal Relic'}
2450,Отчаянный союз,1826,1,4,The Desperate Alliance,[984],UNIQUE,NaN,{'base': 'Vase Relic'}
2451,Последнее пламя,1830,4,1,The Last Flame,[984],UNIQUE,NaN,{'base': 'Incense Relic'}
2452,Вытяжка миротворца,1825,1,3,The Peacemaker's Draught,[983],UNIQUE,NaN,{'base': 'Amphora Relic'}


In [50]:
combined_filtered["tags"] = combined_filtered["tags"].apply(lambda x: [tags_lookup[t] for t in x])
combined_filtered

,name,internal_index,w,h,refName,tags,namespace,category,unique
0,Алый амулет,2381,1,1,Crimson Amulet,[],ITEM,Amulet,NaN
1,Золотой амулет,2390,1,1,Gold Amulet,[],ITEM,Amulet,NaN
2,Амулет с перламутром,2391,1,1,Pearlescent Amulet,[],ITEM,Amulet,NaN
3,Лазурный амулет,2382,1,1,Azure Amulet,[],ITEM,Amulet,NaN
4,Амулет с янтарём,2383,1,1,Amber Amulet,[],ITEM,Amulet,NaN
...,...,...,...,...,...,...,...,...,...
2449,Сменяющиеся сезоны,1827,2,1,The Changing Seasons,[small_sanctum_relic],UNIQUE,NaN,{'base': 'Seal Relic'}
2450,Отчаянный союз,1826,1,4,The Desperate Alliance,[large_sanctum_relic],UNIQUE,NaN,{'base': 'Vase Relic'}
2451,Последнее пламя,1830,4,1,The Last Flame,[large_sanctum_relic],UNIQUE,NaN,{'base': 'Incense Relic'}
2452,Вытяжка миротворца,1825,1,3,The Peacemaker's Draught,[medium_sanctum_relic],UNIQUE,NaN,{'base': 'Amphora Relic'}


In [ ]:
from services.nd_builder_service import NdBuilderService
nd = NdBuilderService(RUSSIAN, "new")
stats = nd.build_stats_ndjson()
items = nd.build_items_ndjson()
items

2025-09-28 09:14:50,621 - services.image_provider - INFO - Saving cache to C:\Dev\ee2-data-builder\src\services/itemImageCache.old.json
2025-09-28 09:14:50,627 - services.nd_builder_service - INFO - Building stats
2025-09-28 09:14:50,628 - services.nd_builder_service - INFO - Importing trade site data


2025-09-28 09:14:50,952 - services.nd_builder_service - INFO - Importing descriptions
2025-09-28 09:14:51,953 - services.nd_builder_service - INFO - Joining trade site and descriptions
2025-09-28 09:14:52,126 - services.nd_builder_service - INFO - Joining stats with and without values
2025-09-28 09:14:52,432 - services.nd_builder_service - INFO - Filling in missing matchers
2025-09-28 09:14:52,793 - services.nd_builder_service - INFO - Adding logbook factions
2025-09-28 09:14:52,811 - services.nd_builder_service - INFO - Cleaning up stats
2025-09-28 09:14:52,824 - services.nd_builder_service - INFO - Building items
2025-09-28 09:14:52,824 - services.nd_builder_service - INFO - Getting unique items
2025-09-28 09:14:52,914 - services.nd_builder_service - INFO - Getting non-unique items
2025-09-28 09:14:52,988 - services.nd_builder_service - INFO - Applying gem and armour columns
2025-09-28 09:14:53,326 - services.nd_builder_service - INFO - Joining unique and non-unique dfs
2025-09-28 09

,name,refName,namespace,unique,icon,tags,tradeTag,craftable,w,h,armour,gem,rune
1865,Подчиняющая порча,Abiding Hex,GEM,NaN,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,{'category': 'Support Skill Gem'},1,1,NaN,"{'awakened': False, 'transfigured': False}",NaN
1867,Едкость,Acrimony,GEM,NaN,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,{'category': 'Support Skill Gem'},1,1,NaN,"{'awakened': False, 'transfigured': False}",NaN
1868,Гранаты-липучки I,Adhesive Grenades I,GEM,NaN,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,{'category': 'Support Skill Gem'},1,1,NaN,"{'awakened': False, 'transfigured': False}",NaN
1869,Гранаты-липучки II,Adhesive Grenades II,GEM,NaN,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,{'category': 'Support Skill Gem'},1,1,NaN,"{'awakened': False, 'transfigured': False}",NaN
1870,Примесь,Admixture,GEM,NaN,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,{'category': 'Support Skill Gem'},1,1,NaN,"{'awakened': False, 'transfigured': False}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Кровь Ксофа,Xoph's Blood,UNIQUE,{'base': 'Amber Amulet'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,NaN,1,1,NaN,NaN,NaN
73,Бремя страданий,Yoke of Suffering,UNIQUE,{'base': 'Bloodstone Amulet'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,NaN,1,1,NaN,NaN,NaN
294,Воспитание Ириэла,Yriel's Fostering,UNIQUE,{'base': 'Strider Vest'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[dex_armour],NaN,NaN,2,3,NaN,NaN,NaN
74,Зарождение Зерфи,Zerphi's Genesis,UNIQUE,{'base': 'Heavy Belt'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,NaN,2,1,NaN,NaN,NaN


In [52]:
s = ref_ts.static()
s["icon_s"] = s["image"].apply(lambda x: f"https://web.poecdn.com{x}")
item_aaa = items.merge(s[["text", "icon_s", "tradeTag"]], left_on="refName", right_on="text", how="left")
item_aaa.loc[item_aaa["icon_s"].notna(), "icon"] = item_aaa["icon_s"]
item_aaa.drop(columns=["text","icon_s"])

,name,refName,namespace,unique,icon,tags,tradeTag_x,craftable,w,h,armour,gem,rune,tradeTag_y
0,Подчиняющая порча,Abiding Hex,GEM,NaN,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,{'category': 'Support Skill Gem'},1,1,NaN,"{'awakened': False, 'transfigured': False}",NaN,NaN
1,Едкость,Acrimony,GEM,NaN,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,{'category': 'Support Skill Gem'},1,1,NaN,"{'awakened': False, 'transfigured': False}",NaN,NaN
2,Гранаты-липучки I,Adhesive Grenades I,GEM,NaN,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,{'category': 'Support Skill Gem'},1,1,NaN,"{'awakened': False, 'transfigured': False}",NaN,NaN
3,Гранаты-липучки II,Adhesive Grenades II,GEM,NaN,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,{'category': 'Support Skill Gem'},1,1,NaN,"{'awakened': False, 'transfigured': False}",NaN,NaN
4,Примесь,Admixture,GEM,NaN,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,{'category': 'Support Skill Gem'},1,1,NaN,"{'awakened': False, 'transfigured': False}",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,Кровь Ксофа,Xoph's Blood,UNIQUE,{'base': 'Amber Amulet'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,NaN,1,1,NaN,NaN,NaN,NaN
2453,Бремя страданий,Yoke of Suffering,UNIQUE,{'base': 'Bloodstone Amulet'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,NaN,1,1,NaN,NaN,NaN,NaN
2454,Воспитание Ириэла,Yriel's Fostering,UNIQUE,{'base': 'Strider Vest'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[dex_armour],NaN,NaN,2,3,NaN,NaN,NaN,NaN
2455,Зарождение Зерфи,Zerphi's Genesis,UNIQUE,{'base': 'Heavy Belt'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],NaN,NaN,2,1,NaN,NaN,NaN,NaN


In [53]:
import json
items_o = items.where(items.notna(), None)
items_o

,name,refName,namespace,unique,icon,tags,tradeTag,craftable,w,h,armour,gem,rune
1865,Подчиняющая порча,Abiding Hex,GEM,None,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],None,{'category': 'Support Skill Gem'},1,1,None,"{'awakened': False, 'transfigured': False}",None
1867,Едкость,Acrimony,GEM,None,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],None,{'category': 'Support Skill Gem'},1,1,None,"{'awakened': False, 'transfigured': False}",None
1868,Гранаты-липучки I,Adhesive Grenades I,GEM,None,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],None,{'category': 'Support Skill Gem'},1,1,None,"{'awakened': False, 'transfigured': False}",None
1869,Гранаты-липучки II,Adhesive Grenades II,GEM,None,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],None,{'category': 'Support Skill Gem'},1,1,None,"{'awakened': False, 'transfigured': False}",None
1870,Примесь,Admixture,GEM,None,https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],None,{'category': 'Support Skill Gem'},1,1,None,"{'awakened': False, 'transfigured': False}",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Кровь Ксофа,Xoph's Blood,UNIQUE,{'base': 'Amber Amulet'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],None,None,1,1,None,None,None
73,Бремя страданий,Yoke of Suffering,UNIQUE,{'base': 'Bloodstone Amulet'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],None,None,1,1,None,None,None
294,Воспитание Ириэла,Yriel's Fostering,UNIQUE,{'base': 'Strider Vest'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[dex_armour],None,None,2,3,None,None,None
74,Зарождение Зерфи,Zerphi's Genesis,UNIQUE,{'base': 'Heavy Belt'},https://web.poecdn.com/gen/image/WzI1LDE0LHsiZ...,[],None,None,2,1,None,None,None


In [55]:
from typing import cast

from constants.filenames import SOUL_CORES, SOUL_CORES_PER_CLASS, STATS
from services.specific_column_service import GENERAL_CLASS_TO_ITEM_CLASS

def first_non_negated(matchers: list[dict[str, str | int | bool]]):
    """
    Returns the first dictionary in 'matchers' list where 'negated' is either absent or False.
    :param matchers: List of dictionaries to search.
    :return: The first non-negated dictionary or None if all are negated.
    """
    sorted_matchers = sorted(
        matchers,
        key=lambda x: len(x["string"]) if isinstance(x["string"], str) else 9999999,
    )
    for matcher in sorted_matchers:
        # Check for absence of 'negated' key or its value being False
        if not matcher.get("negate", False) and not matcher.get("value"):
            return matcher
    for matcher in sorted_matchers:
        if not matcher.get("value"):
            return matcher
    print(sorted_matchers)
    raise ValueError("No non-negated matcher found")

In [56]:
local_runes = ref_gs.get(SOUL_CORES).drop(
        ["_index", "RequiredLevel"], axis=1
    )
stats_lookup: dict[int, str] = ref_gs.get(STATS)["Id"].to_dict()  # pyright:ignore [reportUnknownVariableType,reportUnknownMemberType]

def replace_indices_with_ids(index_list: list[int]):
    return [stats_lookup[i] for i in index_list]

local_runes["StatsArmour"] = local_runes["StatsArmour"].apply(replace_indices_with_ids)  # pyright:ignore [reportUnknownMemberType]
local_runes["StatsMartialWeapon"] = local_runes["StatsMartialWeapon"].apply(  # pyright:ignore [reportUnknownMemberType]
    replace_indices_with_ids
)
local_runes["StatsCasterWeapon"] = local_runes["StatsCasterWeapon"].apply(  # pyright:ignore [reportUnknownMemberType]
    replace_indices_with_ids
)

local_runes["StatsAllEquipment"] = local_runes["StatsAllEquipment"].apply(  # pyright:ignore [reportUnknownMemberType]
    replace_indices_with_ids
)

def process_row(rune: dict[str, int | list[int]], ref_df: pd.DataFrame):
    rune_out: list[
        dict[str, str | int | bool | list[int] | list[str] | None]
    ] = []

    def process_stat(stat_list: list[int], value_list_key: str, key: str):
        if len(stat_list) > 0:
            stat_id = stat_list[0]
            # Look up the row in ref_df with the corresponding 'id' and get 'matchers' and 'trade'
            row = ref_df.loc[ref_df["id"] == stat_id]
            if not row.empty:
                matchers = cast(
                    list[dict[str, str | int | bool]], row["matchers"].iloc[0]
                )
                trade = (
                    cast(dict[str, dict[str, list[str]]], row["trade"].iloc[0])
                    if "trade" in row.columns
                    else {}
                )

                # Translate and extract values
                translated = first_non_negated(matchers).get("string")
                trade_id = (
                    (trade.get("ids") or {}).get("rune") if trade else None
                )

                # Fill the rune_out dictionary
                rune_out.append(
                    {
                        "categories": GENERAL_CLASS_TO_ITEM_CLASS.get(key, []),
                        "string": translated,
                        "values": cast(list[int], rune[value_list_key]),
                        "tradeId": trade_id,
                    }
                )

    if not (
        isinstance(rune["StatsArmour"], list)
        and isinstance(rune["StatsMartialWeapon"], list)
        and isinstance(rune["StatsCasterWeapon"], list)
        and isinstance(rune["StatsAllEquipment"], list)
    ):
        raise ValueError("Stats should be lists")

    # Process each type of stat
    process_stat(rune["StatsArmour"], "StatsValuesArmour", "armour")
    process_stat(
        rune["StatsMartialWeapon"], "StatsValuesMartialWeapon", "weapon"
    )
    process_stat(rune["StatsCasterWeapon"], "StatsValuesCasterWeapon", "caster")

    for this_key in GENERAL_CLASS_TO_ITEM_CLASS.keys():
        process_stat(
            rune["StatsAllEquipment"], "StatsValuesAllEquipment", this_key
        )

    return rune_out

local_runes["rune"] = local_runes.apply(process_row, axis=1, args=(stats,))
filtered_runes = local_runes[["BaseItemType", "rune"]]
runes_a = filtered_runes[
    filtered_runes["rune"].apply(lambda x: len(x) > 0)  # pyright:ignore [reportUnknownMemberType,reportUnknownArgumentType,reportUnknownLambdaType]
]


In [57]:
local_runes_two = ref_gs.get(SOUL_CORES_PER_CLASS).drop(["_index"], axis=1)
classes: dict[int, int] = ref_gs.get(ITEM_CLASSES)["Id"].to_dict()  # pyright:ignore [reportUnknownVariableType,reportUnknownMemberType]
stats_lookup: dict[int, str] = ref_gs.get(STATS)["Id"].to_dict()  # pyright:ignore [reportUnknownVariableType,reportUnknownMemberType]

def replace_indices_with_ids(index_list: list[int]):
    return [stats_lookup[i] for i in index_list]

local_runes_two["Stats"] = local_runes_two["Stats"].apply(replace_indices_with_ids)  # pyright:ignore [reportUnknownMemberType]

def process_row(rune: dict[str, int | list[int]], ref_df: pd.DataFrame):
    rune_out: list[
        dict[str, str | int | bool | list[int] | list[str] | None]
    ] = []

    if not (isinstance(rune["Stats"], list)):
        raise ValueError("Stats should be lists")

    if len(rune["Stats"]) == 0:
        return rune_out

    stat_id = rune["Stats"][0]
    # Look up the row in ref_df with the corresponding 'id' and get 'matchers' and 'trade'
    row = ref_df.loc[ref_df["id"] == stat_id]
    if not row.empty:
        matchers = cast(
            list[dict[str, str | int | bool]], row["matchers"].iloc[0]
        )
        trade = (
            cast(dict[str, dict[str, list[str]]], row["trade"].iloc[0])
            if "trade" in row.columns
            else {}
        )

        # Translate and extract values
        translated = first_non_negated(matchers).get("string")
        trade_id = (trade.get("ids") or {}).get("rune") if trade else None
        if not isinstance(rune["ItemClass"], int):
            raise ValueError("ItemClass should be an integer")

        # Fill the rune_out dictionary
        rune_out.append(
            {
                "categories": classes[rune["ItemClass"]],
                "string": translated,
                "values": cast(list[int], rune["StatsValues"]),
                "tradeId": trade_id,
            }
        )

    return rune_out

local_runes_two["rune"] = local_runes_two.apply(process_row, axis=1, args=(stats,))
filtered_runes = local_runes_two[["BaseItemType", "rune"]]
runes_b = filtered_runes[
    filtered_runes["rune"].apply(lambda x: len(x) > 0)  # pyright:ignore [reportUnknownMemberType,reportUnknownArgumentType,reportUnknownLambdaType]
]

In [58]:

runes = pd.concat([runes_a, runes_b])

grouped = runes.groupby("BaseItemType", as_index=False).agg(  # pyright:ignore [reportUnknownMemberType]
    {"rune": lambda runes: [r for rune_list in runes for r in rune_list]}  # pyright:ignore [reportUnknownLambdaType,reportUnknownVariableType]
)

rune_ad = items.merge(
    grouped,
    left_on="internal_index",
    right_on="BaseItemType",
    how="left",
)
rune_ad.loc[rune_ad["refName"].str.startswith("Great")]

KeyError: 'internal_index'

In [ ]:
with open(f"output_i.ndjson", "w", encoding="utf-8") as f:
    for _, row in items_o.iterrows():
        obj = {k: v for k, v in row.items() if v is not None}
        f.write(f"{json.dumps(obj, ensure_ascii=False, default=int)}\n")